# Explorar a Camada Silver

In [ ]:
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates
import pyspark.pandas as ps
#
# Nome da aplicação Spark
#
APP_NAME="ExplorarSilver"
ps.options.display.max_rows = 10

## Inicializar Spark e definir funções de apoio

In [ ]:
%run StartSpark.ipynb

## Carregar as tabelas da camada Silver

In [ ]:
_ = loadAndRegister("stocks", "StockData", "silver")

In [ ]:
%%sql_display
select
    ticker,
    day,
    count(*)
from silver_stocks
group by all
order by ticker, day

In [ ]:
%%sql_display
select
    ticker,
    day,
    timestamp,
    open,
    high,
    low,
    close
from silver_stocks
where ticker="AAPL"
 and day = "2024-05-22"
order by ticker, timestamp
limit 1

In [ ]:
%%sql_display
select
    ticker,
    day,
    timestamp,
    open,
    high,
    low,
    close
from silver_stocks
where ticker="AAPL"
 and day = "2024-05-22"
order by ticker, timestamp desc
limit 1

In [ ]:
%%sql_display
CREATE OR REPLACE TEMPORARY VIEW stocks_data AS 
select * 
from silver_stocks 
order by ticker, day, timestamp

In [ ]:
%%sql_display
show tables

In [ ]:
%%sql_display
CREATE OR REPLACE TEMPORARY VIEW gold_daily AS
select
    ticker,
    day,
    first(open) as open,
    min(low) as low,
    max(high) as high,
    last(close) as close,
    sum(volume) as volume
from stocks_data
group by all
order by ticker, day

In [ ]:
df = %sql select *, (close - open) as osc, (osc / open) as osc_per from gold_daily

In [ ]:
df.pandas_api()

In [ ]:
# (df
#   .write
#   .format("jdbc")
#   .option("url", postgresUrl)
#   .option("driver", "org.postgresql.Driver")
#   .option("dbtable", "stocks_daily")
#   .option("user", postgresUsername)
#   .option("password", postgresPassword)
#   .mode("overwrite")
#   .save()
# )

In [ ]:
df = %sql \
select \
    _id, \
    ticker, \
    day, \
    description, \
    timestamp - interval 3 hours as timestamp, \
    open, \
    high, \
    low, \
    close, \
    volume, \
    osc, \
    osc_per, \
    __op, \
    __collection, \
    __ts_ms \
from silver_stocks \
order by ticker, timestamp desc

In [ ]:
df.pandas_api()

In [ ]:
# (df
#   .write
#   .format("jdbc")
#   .option("url", postgresUrl)
#   .option("driver", "org.postgresql.Driver")
#   .option("dbtable", "stocks_intraday")
#   .option("user", postgresUsername)
#   .option("password", postgresPassword)
#   .mode("overwrite")
#   .save()
# )